In [ ]:
"""
YKS Analiz Sistemi - Ana Notebook
Tüm modüllerin entegre kullanımı
"""

# ============================================================================
# CELL 1: Kütüphaneleri İçe Aktar
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Kendi modüllerimizi içe aktar
from data_loader import GoogleSheetsLoader
from data_cleaner import DataCleaner
from analysis.net_analyzer import NetAnalyzer
from analysis.topic_analyzer import TopicAnalyzer
from visualization.net_charts import NetVisualizer
from visualization.topic_charts import TopicVisualizer
from config import Config
from dotenv import load_dotenv

print("✅ Tüm modüller yüklendi!")



In [ ]:
# CELL 2: Konfigürasyon Ayarları

# .env dosyasını bu hücrede en başta yükle
load_dotenv()


config = Config()

# validate_config artık bir static method olduğu için bu şekilde çağrılabilir.
Config.validate_config()

# Ortam değişkenlerinden değerleri al
SHEET_URL = os.getenv("GOOGLE_SHEET_URL")
CREDENTIALS_PATH = os.getenv("CREDENTIALS_PATH")
print("📄 Google Sheets URL:", SHEET_URL)
print("🔑 Credentials Path:", CREDENTIALS_PATH)

EXAM_TYPE = "TYT"
TARGET_NET = config.Analysis.DEFAULT_TARGET_NET[EXAM_TYPE]

print(f"📊 Sınav Türü: {EXAM_TYPE}")
print(f"🎯 Hedef Net: {TARGET_NET}")

# loader'a ortam değişkenlerinden alınan değerleri ver
loader = GoogleSheetsLoader(SHEET_URL, CREDENTIALS_PATH)
cleaner = DataCleaner(config, strict_mode=False, auto_fix=True)
net_analyzer = NetAnalyzer(config, exam_type=EXAM_TYPE)
topic_analyzer = TopicAnalyzer(config, exam_type=EXAM_TYPE)
net_viz = NetVisualizer(config)
topic_viz = TopicVisualizer(config)

print("\n✅ Tüm ana nesneler yapılandırma ile oluşturuldu!")

In [ ]:
print("\n📥 Veriler yükleniyor...")

if EXAM_TYPE == "TYT":
    raw_data = loader.load_tyt_data()
else:
    raw_data = loader.load_ayt_data()

print(f"✅ {len(raw_data)} deneme yüklendi")
print("\n📋 Yüklenen Ham Veriden İlk 3 Satır:")
display(raw_data.head(3))

In [ ]:
# ============================================================================
# CELL 4: Veri Temizleme
# ============================================================================

print("\n🧹 Veriler temizleniyor...")

# Temizleyici oluştur
cleaner = DataCleaner(config, strict_mode=False, auto_fix=True)

# Tam temizleme
cleaned_data = cleaner.clean_full_dataset(raw_data, exam_type=EXAM_TYPE)

# Türetilmiş özellikler ekle
cleaned_data = cleaner.add_derived_features(cleaned_data, exam_type=EXAM_TYPE)

# Temizleme raporu
report = cleaner.get_cleaning_report()
print(f"\n📊 Temizleme Raporu:")
print(f"  • Çıkarılan satır: {report['rows_removed']}")
print(f"  • Düzeltilen değer: {report['values_fixed']}")
print(f"  • Uyarılar: {len(report['warnings'])}")

print(f"\n✅ Temizleme tamamlandı: {len(cleaned_data)} deneme")



In [ ]:
# ============================================================================
# CELL 5: Net Analizi
# ============================================================================

print("\n📊 Net analizi yapılıyor...")

# Net analizör oluştur
net_analyzer = NetAnalyzer(config, exam_type=EXAM_TYPE)

# Toplam net istatistikleri
print("\n=== TOPLAM NET İSTATİSTİKLERİ ===")
total_stats = net_analyzer.calculate_statistics(cleaned_data, 'Toplam Net')
for key, value in total_stats.items():
    print(f"  {key}: {value:.2f}" if isinstance(value, float) else f"  {key}: {value}")

# Trend analizi
print("\n=== TREND ANALİZİ ===")
trend = net_analyzer.get_progression_trend(cleaned_data, 'Toplam Net')
print(f"  Trend: {trend['trend']}")
print(f"  Eğim: {trend['slope']:.2f} net/deneme")
print(f"  R²: {trend['r_squared']:.3f}")
print(f"  Toplam İlerleme: {trend['total_improvement']:.1f} net")
print(f"  Sonraki Tahmin: {trend['next_prediction']:.1f} net")

# Tüm dersler için istatistikler
print("\n=== DERS BAZLI İSTATİSTİKLER ===")
all_stats = net_analyzer.get_all_subjects_statistics(cleaned_data)
print(all_stats[['mean', 'std', 'min', 'max']].round(2))

# Zayıf ve güçlü dersler
print("\n=== ZAYIF DERSLER ===")
weak = net_analyzer.identify_weak_subjects(cleaned_data)
for subject_info in weak[:3]:
    print(f"  • {subject_info['subject']}: {subject_info['mean']:.1f} net (Trend: {subject_info['trend']})")

print("\n=== GÜÇLÜ DERSLER ===")
strong = net_analyzer.identify_strong_subjects(cleaned_data)
for subject_info in strong[:3]:
    print(f"  • {subject_info['subject']}: {subject_info['mean']:.1f} net (Trend: {subject_info['trend']})")

# Hedefe göre durum
print("\n=== HEDEFE GÖRE DURUM ===")
target_comparison = net_analyzer.compare_to_target(cleaned_data, TARGET_NET)
print(f"  Hedef: {target_comparison['target']}")
print(f"  Mevcut: {target_comparison['current']:.1f}")
print(f"  Fark: {target_comparison['gap']:.1f} net")
print(f"  Durum: {target_comparison['status']}")
if target_comparison['exams_needed']:
    print(f"  Tahmini Deneme Sayısı: {target_comparison['exams_needed']}")



In [ ]:
# ============================================================================
# CELL 6: Konu Analizi
# ============================================================================

print("\n📚 Konu analizi yapılıyor...")


topic_analyzer = TopicAnalyzer(config, exam_type=EXAM_TYPE)
# Ön hesaplama işlemi
precomputed_topic_trends = topic_analyzer._precompute_all_topic_trends(cleaned_data)


# En çok yanlış yapılan konular
print("\n=== EN ÇOK YANLIŞ YAPILAN 10 KONU ===")
problematic = topic_analyzer.get_most_problematic_topics(cleaned_data, top_n=10)
for i, (topic, count) in enumerate(problematic, 1):
    print(f"  {i}. {topic}: {count} kez")

# Zayıf alanlar (ders bazlı)
print("\n=== ZAYIF ALANLAR (DERS BAZLI) ===")
weak_areas = topic_analyzer.identify_weak_areas(cleaned_data, threshold=3)
for subject, topics in weak_areas.items():
    print(f"\n{subject}:")
    for topic in topics[:5]:  # İlk 5 konu
        print(f"  • {topic}")

# Çalışma planı (Değişken adı ve "..." hatası düzeltildi)
print("\n=== ÖNERİLEN ÇALIŞMA PLANI ===")
study_plan = topic_analyzer.generate_study_plan(
    cleaned_data, 
    precomputed_topic_trends, # Değişken adı düzeltildi
    max_topics_per_subject=3
)
for subject, plan in study_plan.items():
    print(f"\n{subject}:")
    for item in plan:
        print(f"  {item['sıra']}. {item['konu']}")
        print(f"     Öncelik: {item['öncelik']} | Frekans: {item['frekans']} | {item['son_durum']}")

# Ders karşılaştırması
print("\n=== DERS BAZLI KONU KARŞILAŞTIRMASI ===")
subject_comparison = topic_analyzer.compare_subjects_by_topics(cleaned_data)
print(subject_comparison)

In [ ]:
# ============================================================================
# CELL 7: Net Grafikleri
# ============================================================================

# Net Analiz Sonuçları
all_net_stats = net_analyzer.get_all_subjects_statistics(cleaned_data)
improvement_data = net_analyzer.calculate_improvement_rate(cleaned_data, window=3) # Veya config'den al

# Konu Analiz Sonuçları
most_problematic = topic_analyzer.get_most_problematic_topics(cleaned_data, top_n=15)
subject_comparison_topics = topic_analyzer.compare_subjects_by_topics(cleaned_data)


print("\n📈 Net grafikleri oluşturuluyor...")

# Görselleştirici oluştur
net_viz = NetVisualizer(config)

# Klasör oluştur
charts_dir = Path("output/charts")
charts_dir.mkdir(parents=True, exist_ok=True)

# 1. Toplam Net Trendi
fig1 = net_viz.plot_total_nets_by_exam(
    cleaned_data, 
    save_path=charts_dir / "01_toplam_net.png"
)
plt.show()

# 2. Tüm Derslerin Karşılaştırması (Son Deneme)
fig2 = net_viz.plot_all_subjects_comparison(
    cleaned_data,
    save_path=charts_dir / "02_ders_karsilastirma.png"
)
plt.show()

net_columns = [col for col in cleaned_data.columns if col.endswith('Net')]

# 4. Çok Çizgili Karşılaştırma
fig4 = net_viz.plot_multi_subject_comparison(
    cleaned_data,
    net_columns[:4] if len(net_columns) >= 4 else net_columns,
    save_path=charts_dir / "04_coklu_karsilastirma.png"
)
plt.show()

print(f"✅ Net grafikleri kaydedildi: {charts_dir}")



In [ ]:
# ============================================================================
# CELL 8: Konu Grafikleri
# ============================================================================

print("\n📊 Konu grafikleri oluşturuluyor...")

# Konu görselleştiricisi
topic_viz = TopicVisualizer(config)

# Son 3 deneme için en çok yanlış yapılan konular
last3_data = cleaned_data.sort_values('Tarih').tail(3)
problematic_last3 = topic_analyzer.get_most_problematic_topics(last3_data, top_n=15)

fig8 = topic_viz.plot_total_wrong_topics(
    problematic_topics=problematic_last3, 
    top_n=15,
    save_path=charts_dir / "08_en_cok_yanlis.png",
)
plt.show()
net_viz.dashboard(cleaned_data, exam_type=EXAM_TYPE)

fig = topic_viz.plot_topic_trend_by_exam(cleaned_data, 'Matematik', 'türev')
if fig:
    plt.show()
else:
    print("Grafik için yeterli veri yok veya isimler eşleşmiyor.")


print(f"✅ Konu grafikleri kaydedildi: {charts_dir}")

In [ ]:
# ============================================================================
# CELL 9: Final Summary Report
# ============================================================================

# Kapsamlı rapor oluştur
net_report = net_analyzer.generate_summary_report(cleaned_data)
topic_report = topic_analyzer.generate_topic_summary_report(cleaned_data)

print(f"\n📊 Genel Bilgiler:")
print(f"  • Sınav Türü: {EXAM_TYPE}")
print(f"  • Toplam Deneme Sayısı: {net_report.get('total_exams', 'N/A')}")
print(f"  • Çalışılan Tarih Aralığı: {net_report.get('date_range', {}).get('first', pd.NaT).strftime('%Y-%m-%d')} -> {net_report.get('date_range', {}).get('last', pd.NaT).strftime('%Y-%m-%d')}")

print(f"\n🎯 Net Performansı:")
print(f"  • Ortalama Toplam Net: {net_report['overall_stats'].get('mean', 0):.2f}")
print(f"  • En Yüksek Toplam Net: {net_report['overall_stats'].get('max', 0):.2f}")
print(f"  • Son Deneme Toplam Net: {net_report['overall_stats'].get('latest', 0):.2f}")
print(f"  • İlerleme: {net_report['recent_improvement'].get('interpretation', 'N/A')}")

print("\n🎉 Analiz tamamlandı!")

In [ ]:
# ============================================================================
# CELL 10: Veriyi Kaydet (Opsiyonel)
# ============================================================================

# Temizlenmiş veriyi CSV olarak kaydet
output_data_dir = Path("output/data")
output_data_dir.mkdir(parents=True, exist_ok=True)

cleaned_data.to_csv(output_data_dir / f"{EXAM_TYPE}_temiz_veri.csv", index=False)
print(f"\n💾 Temizlenmiş veri kaydedildi: {output_data_dir / f'{EXAM_TYPE}_temiz_veri.csv'}")

# Özet raporları JSON olarak kaydet
import json

with open(output_data_dir / f"{EXAM_TYPE}_net_rapor.json", 'w', encoding='utf-8') as f:
    # DataFrame'leri dict'e çevir
    report_to_save = net_report.copy()
    for key in report_to_save:
        if isinstance(report_to_save[key], pd.DataFrame):
            report_to_save[key] = report_to_save[key].to_dict()
    json.dump(report_to_save, f, ensure_ascii=False, indent=2, default=str)

print(f"💾 Net raporu kaydedildi: {output_data_dir / f'{EXAM_TYPE}_net_rapor.json'}")

print("\n🎉 Tüm işlemler tamamlandı!")